# Docker "Swarm Mode" Lab

Originally based on Mario Loriedo's gist here: https://gist.github.com/l0rd/5186cc80f8f26dc7e9490abca4405830 and extended

# Requirements
- Docker 1.12+
- Docker machine [[RELEASES]](https://github.com/docker/machine/releases)
- Virtualbox [[virtualbox.org]](https://www.virtualbox.org/)

# Create 3 nodes for your swarm cluster (one master and 2 slaves)

We will create 3 nodes using docker-machine/virtualbox.

![](images/SwarmNodes_3nodes.png)

We will then join them to form a Swarm cluster:
![](images/SwarmNodes_3nodes_1m_2w.png)




#### NOTE:
You may see errors as below, and an error reported by "docker-machine ls", leave some time for the swmaster1 to settle.

In the meantime you can go ahead and create the 2 swnode's below.

In [1]:
MACHINE_DRIVER=virtualbox

CLEANUP_VMS=1
CLEANUP_VMS=0
CLEANUP_VMS=1

. ../NB_bash_functions.rc

Notebook started at Wed Oct 25 10:42:39 CEST 2017   [1508920959]


In [ ]:
# Specific to my environment

[ -f ~/.docker.rc ] && . ~/.docker.rc

NB_continue

In [ ]:
MACHINE_DRIVER=virtualbox

#MACHINE_DRIVER=digitalocean
#MACHINE_DRIVER=azure
#MACHINE_DRIVER=google

Let's cleanup any remaining machines, if they exist already ..

In [ ]:
VBoxManage list runningvms | grep sw

echo
docker-machine ls

In [ ]:
docker-machine ls --filter name=sw --format "{{.Name}}: {{.DriverName}}"

In [ ]:
[ $CLEANUP_VMS -ne 0 ] && NB_cleanup_swarm_machines

NB_continue

In [ ]:
[ $CLEANUP_VMS -ne 0 ] &&  {
    VBoxManage list runningvms | grep sw

    echo
    docker-machine ls
}

NB_continue

Now let's create our new nodes

**NOTE:** You may see some worrying messages from "*docker-machine ls*" but wait a minute or two until the node creation is complete

In [ ]:
NB_create_swarm_machine_m1

#### NOTE: Temporary errors from "*docker-machine ls*"

From the command-line you can see the progress of machine creation, using
    ```$ docker-machine ls```
    
You may see errors during creation of machines before ssh connectivity is established and before docker host is started.

![](images/docker-machine-errors.png)

After creation of a node you should see something like the following from "*docker-machine ls*"

In [ ]:
docker-machine ls

We can also see the VM from virtualbox point of view (if this is the driver we are using):

In [ ]:
VBoxManage list runningvms | grep sw

In [ ]:
NB_create_swarm_machine_1

In [ ]:
VBoxManage list runningvms | grep sw

echo
docker-machine ls

In [ ]:
NB_create_swarm_machine_2

In [ ]:
VBoxManage list runningvms | grep sw

echo
docker-machine ls

In [ ]:
NB_check_3_machines_running

### We have now created 3 docker hosts, which are operating independently for the moment:


![](images/SwarmNodes_3nodes.png)



### Directing the docker client to a particular nodes' docker daemon

**NOTE**: See that we precede all docker commands with $(docker-machine config NODE) where node is the name of the node to which we want our docker client to connect to.  This command returns the parameters to direct our client to the appropriate node.  Run alone this produces:

In [ ]:
docker-machine config swmaster1

We can also obtain these values as environment variables

In [ ]:
docker-machine env swmaster1

Including these parameters on the docker command line will connect the client to the docker daemon running on node '*swmaster1*'.

In [ ]:
docker-machine ssh swmaster1 "hostname; uptime; docker version"

# Creating the swarm cluster

We now want to move our 3 nodes from being independent:
![](images/SwarmNodes_3nodes.png)

to being 1 Master and 2 Worker Swarm Nodes:
![](images/SwarmNodes_3nodes_1m_2w.png)



### Networks before creation of swarm cluster
Before going further let's look at the networks on your machine.

Later, we'll see how a new network is created once the swarm cluster has been created.

In [ ]:
docker $(docker-machine config swmaster1) network ls

Note that we already see **3 networks** each of a different **type**:
- **host**: this is the network of your host machine (the network on your host swmaster1)
- **bridge**: this is a separate network on which containers will have their own ip, separated from the host network
- **none/null**:

Now let's identify the ip address of our master node.

We can see this through config or ip commands of docker-machine as shown below.

In [ ]:
docker-machine ip swmaster1

We could then provide the above ip address as parameter to --advertise-addr when initializing the swarm.

However, it is quite convenient to run the above commands embedded, as below, as arguments to the swarm init command.

docker-machine config swmaster1 provides the parameters to use when connecting to the appropriate docker engine for our machine "swmaster1".

The following command will run swarm init to generate the cluster with 'swmaster1' as the Master node.
You should see output similar to the below:

In [ ]:
docker $(docker-machine config swmaster1) info | grep Swarm

# swarm init

Now that we have 3 nodes available, we will initialize our Swarm Cluster with 1 master node.

In [ ]:
which docker

In [ ]:
docker $(docker-machine config swmaster1) swarm init --advertise-addr $(docker-machine ip swmaster1)

![](images/SwarmNodes_3nodes_1m.png)

A docker info should now show "Swarm: active" as below:

In [ ]:
docker $(docker-machine config swmaster1) info | grep Swarm:

In [ ]:
docker $(docker-machine config swmaster1) info

If we look at the networks we should now see new networks such as '*ingress*' an overlay network and docker_gwbridge for the swarm cluster.

In [ ]:
docker $(docker-machine config swmaster1) network ls

We see that 2 new networks have been created of type bridge and overlay
- The **ingress** network is a special overlay network that facilitates **load balancing** among a service’s nodes
- The **docker_gwbridge** is a bridge network that connects the overlay networks (including the ingress network) to an individual Docker daemon’s physical network.

By default, each container a service is running is connected to its local Docker daemon host’s docker_gwbridge network.

# swarm join

Now we wish to join Master and Worker nodes to our swarm cluster, to do this we need to obtain the token generated during the "swarm init".

Let's save that token to an environment variable as follows:

In [ ]:
worker_token=$(docker $(docker-machine config swmaster1) swarm join-token worker -q)

In [ ]:
echo $worker_token

Now we can use this token to join nodes as a worker to this cluster

Note: we could also join nodes as Master, but we have only 3 nodes available.

Let's join swnode1 as a worker node

In [ ]:
docker $(docker-machine config swnode1) swarm join --token $worker_token $(docker-machine ip swmaster1):2377

![](images/SwarmNodes_3nodes_1m_1w.png)

Now we can use the same token to join node swnode2 as a worker node


In [ ]:
docker $(docker-machine config swnode2) swarm join --token $worker_token $(docker-machine ip swmaster1):2377

![](images/SwarmNodes_3nodes_1m_2w.png)


In [ ]:
docker $(docker-machine config swmaster1) node ls

# start service

First we check for any running services - there should be none in our newly initialized cluster:

In [ ]:
docker $(docker-machine config swmaster1) service ls

Now we will create a new service based on the docker image mjbright/docker-demo

We will expose this service on port 8080


In [ ]:
docker $(docker-machine config swmaster1) service create --detach --replicas 1 --name docker-demo -p 8080:8080 mjbright/docker-demo:20

Now we list services again and we should see our newly added docker-demo service

In [ ]:
docker $(docker-machine config swmaster1) service ls

In [ ]:
docker $(docker-machine config swmaster1) service ls

In [ ]:
NB_docker_loop_until_service_started docker-demo

it may take a few seconds until the service has a running container (see REPLICAS column) especially if the container image has not yet been downloaded

In [ ]:
docker $(docker-machine config swmaster1) service ls

... and we can look at the service as seen by the cluster:

In [ ]:
docker $(docker-machine config swmaster1) service ps docker-demo

... and we can look at the service on the individual cluster nodes.

Of course as we set replicas to 1 there is only 1 replica of the service for the moment, running on just 1 node of our cluster:

In [ ]:
docker $(docker-machine config swmaster1) ps

In [ ]:
docker $(docker-machine config swnode1) ps

In [ ]:
docker $(docker-machine config swnode2) ps

# Accessing the service

As we are working remotely we need to create an ssh tunnel through to the swarm cluster to access our service, exposing the port 8080 on your local machine.

We can obtain the ip address of the swarm master node as follows.

In [ ]:
NODE=swnode1

IP=$(docker-machine ip $NODE)

echo $IP

echo "Connect to http://${IP}:8080"

Let's set an environment variable with that ip address

In [ ]:
MASTERIP=$(docker-machine ip swmaster1)

Then open your web browser at the above page [http://IP:8080](http://${IP}:8080) (replace <IP> with the appropriate value) and you should see a lovely blue whale, as below:

![](images/docker.png)


Alternatively we can connect using a command-line client (the web server detects if it's a wget or curl request and returns ASCII text instead of an image).

In [ ]:
wget -O - http://$IP:8080

# scale service

Now we can scale the service to 3 replicas:

In [ ]:
docker $(docker-machine config swmaster1) service scale docker-demo=3

In [ ]:
docker $(docker-machine config swmaster1) service ps docker-demo

# rolling-update

Now we will see how we can perform a rolling update.

We initially deployed version 20 of the service, now we will upgrade our whole cluster to version 21


In [ ]:
docker $(docker-machine config swmaster1) service ps docker-demo

In [ ]:
docker $(docker-machine config swmaster1) service update --image mjbright/docker-demo:21 docker-demo

In [ ]:
docker $(docker-machine config swmaster1) pull mjbright/docker-demo:21

In [ ]:
docker $(docker-machine config swnode1) pull mjbright/docker-demo:21

In [ ]:
docker $(docker-machine config swnode2) pull mjbright/docker-demo:21

In [ ]:
docker $(docker-machine config swmaster1) service ps docker-demo

Again it may take a few moments until the new image is downloaded and operational on all nodes.

Once the new image has been downloaded and started on all nodes you should see something like:

In [ ]:
docker $(docker-machine config swmaster1) service ps docker-demo

Now connect again to

In [ ]:
MASTERIP=$(docker-machine ip swmaster1)
echo "http://${MASTERIP}:8080"

### Verifying the service has been updated

Then open your web browser at the page http://MASTERIP:8080 and you should now see a lovely **red** whale.


![](images/docker_red.png)

of if you prefer the command-line ..

In [ ]:
#. ../NB_bash_functions.rc

NB_docker_loop_until_N_replicas_started docker-demo 3

#NB_pause

In [ ]:
docker $(docker-machine config swmaster1) service ls

In [ ]:
NB_docker_loop_until_service_started docker-demo

In [ ]:
wget -O - http://$MASTERIP:8080

## Service rollback

Revert changes to a service’s configuration

#### API 1.31+
The client and daemon API must both be at least 1.31 to use this command.

Use the docker version command on the client to check your client and daemon API versions.

In [ ]:
docker version

In [ ]:
docker service rollback docker-demo

# drain a node

We can drain a node effectively placing it in 'maintenance mode'.

Draining a node means that it no longer has running tasks on it.

In [ ]:
docker $(docker-machine config swmaster1) node ls

Let's drain swnode1

In [ ]:
docker $(docker-machine config swmaster1) service ps docker-demo

In [ ]:
docker $(docker-machine config swmaster1) node update --availability drain swnode1

and now we see that all services on swnode1 are shutdown

In [ ]:
docker $(docker-machine config swmaster1) service ps docker-demo

But as we had request 3 replicas of our service the Swarm mode will take care to start extra instances on other nodes so that we still have 3 replicas of the service running.

So after a few seconds we see that we once again have 3 replicas in the *Running* state, as shown below:

In [ ]:
docker $(docker-machine config swmaster1) service ps docker-demo

# remove a service

Now let's cleanup by removing our service

In [ ]:
docker $(docker-machine config swmaster1) service rm docker-demo

We can check that the service is no longer running:

In [ ]:
docker $(docker-machine config swmaster1) service ps docker-demo

NB_continue

In [ ]:
docker $(docker-machine config swmaster1) ps

# Deploying a stack

Since Docker 1.12 there is the concept of a Docker Stack.

Docker Stack, similar to Docker-compose allows to deploy a whole application made up of a group of components and especially a group of services.

Stacks extend this concept providing more features such as scaling and node placement.

Let's deploy a stack

First let's clone Alex Ellis' OpenFaaS

(FaaS = Function as a Service - a Docker implementation of "Serverless Computing").

https://github.com/alexellis/faas

There is a quick test drive document here:
https://github.com/alexellis/faas/blob/master/TestDrive.md

from which the following steps were taken:

In [ ]:
git clone https://github.com/alexellis/faas alexellis.faas
cd alexellis.faas

Take the time to look at the docker-compose.yml file.

Note that this is the latest version 3 with extra stack capabilities such as deploy/placement tags.

In [ ]:
cat docker-compose.yml

In [ ]:
cat deploy_stack.sh

**Note: if running on play-with-docker.com**, copy-paste the following directly

```
      docker swarm init --advertise-addr eth0 && \
        git clone https://github.com/alexellis/faas && \
        cd faas && \
        ./deploy_stack.sh && \
        docker service ls
```

**Note**: Running under play-with-docker.com you will have access to Prometheus also

In [ ]:
which docker-machine

docker-machine version

In [ ]:
eval $(docker-machine env swmaster1)

time ./deploy_stack.sh

In [ ]:
git checkout 17cd3d8028f2096fdecc7ea9f5813af17c1bb1cb

In [ ]:
eval $(docker-machine env swmaster1)

time ./deploy_stack.sh

If you are running on PWD, click on the link to port 8080 at the top of the page.

If you are running via docker-machine open an ssh tunnel to forward this port locally
    docker-machine ssh swmaster1 -L 8080:localhost:8080

### Connect to FaaS dashboard
Then connect to http://localhost:8080 to see the FaaS dashboard

In [ ]:
echo "http://${MASTERIP}:8080"

### Inspect your FaaS implementation

From the command-line inspect the stack you've just deployed using
- docker service ls
- docker service ps &lt;service-name&gt;

### Play with FaaS
The point of this exercise was to demonstrate the deploying of the stack, but why not play with FaaS a bit now, you deserve it !

But don't spend too long, you've a Docker-Python class to do next !!

## Visualizing the stack

We can use Mano Marks' visualizer application to view our deployed stack.

Let's launch the visualizer on swmaster1, exposed on port 7070

In [ ]:
docker $(docker-machine config swmaster1) run -it -d -p 7070:8080 -v /var/run/docker.sock:/var/run/docker.sock dockersamples/visualizer

we can now connect to the stack at swmaster1:7070

In [ ]:
MASTERIP=$(docker-machine ip swmaster1)

echo "Connect to http://$MASTERIP:7070"

You should see something like this

![](images/visualizer.png)

In [ ]:
docker $(docker-machine config swmaster1) ps

In [ ]:
docker $(docker-machine config swnode1) ps

In [ ]:
docker $(docker-machine config swnode2) ps

In [ ]:
docker node inspect --pretty swnode1


In [ ]:
docker node update --availability active swnode1

In [ ]:
docker node inspect --pretty swnode1


In [ ]:
docker $(docker-machine config swnode1) ps

In [ ]:
NB_time_taken